# Content Check List

1. Find tickers of current S&P500 companies - Completed
2. From S&P 500 companies, select 5 top companies from each industry sector, based on companys’ market capitalization -- Completed
3. Then rank these 5 top company in their own industry sector respectively based on their most important financial metrics -- Almost Completed (details below)

3.1 Ranks Included

'Financials': Return on Equity (ROE)
'Utilities': Debt-To-Equity (D/E) Ratios
'Health Care': Cash, FCF
'Consumer Staples': EBITDA/EV
'Energy': EBITDA/EV
'Industrials': None
'Materials': None
'Communication Services': EPS growth
'Consumer Discretionary': # Sales growth

3.2 Ranks will be included further and the challenges faced

(1). 'Communication Services': P/E ratio 
problems: I haven't get a way to know how to get historical market value per share 

(2) 'Information Technology': Price to Sales 
problems: don't know how to get historical marketcap

(3) 'Real Estate': FFO | NAV 
problems: don't know how to calculate it

### Import Packages

In [1]:
# pip install yfinance
import yfinance as yf
#!pip install yahooquery
from yahooquery import Ticker
import bs4 as bs
import pandas as pd
import pickle
import requests

In [2]:
# Brad added on 20200820:
sp500_tickers_location = "/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/quant_research/sp500_tickers_updates/"


# Find tickers of current S&P500 companies

In [5]:
# def save_sp500_tickers():
#     """
#     This is a function to save current stock ticker in S&P500
    
#     Input: None
    
#     Output:
#     tickers: A list of ticker name of S&P500 stocks
#     """
#     resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
#     soup = bs.BeautifulSoup(resp.text, 'lxml')
#     table = soup.find('table', {'class': 'wikitable sortable'})
#     ### Collect the Ticker Name from Wikipedia tables
#     tickers = []
#     for row in table.findAll('tr')[1:]:
#         ticker = row.findAll('td')[0].text
#         tickers.append(ticker)
#     ### Remove the newline character from list
#     sample_list = tickers
#     converted_list = []
#     for element in sample_list:
#         converted_list.append(element.strip())
#     ### In Yahoo Finance, the website uses '-' in ticker name instead of '.'
#     ### Replace "." With "-"
#     new_strings = []
#     for string in converted_list:
#         new_string = string.replace(".", "-")
#         new_strings.append(new_string)
#     tickers = new_strings
    
#     """
#     ### Save the Ticker into file
#     with open("sp500tickers.pickle","wb") as f:
#         pickle.dump(tickers,f)    
#     """
#     return tickers

In [6]:
### Call Function
# tickers = save_sp500_tickers()

In [7]:
# print("length of tickers list:", len(set(tickers)))

# From S&P 500 companies, select 5 top companies from each industry sector, based on companys’ market capitalization

### Extract Sector Information

In [4]:
# ### Create a function to extract the "sector" of stocks in S&P 500
# def save_sp500_sector():
#     """
#     This is a function to save current stock sectors in S&P500
    
#     Input: None
    
#     Output:
#     sectors: A list of sector name of S&P500 stocks
#     """
#     resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
#     soup = bs.BeautifulSoup(resp.text, 'lxml')
#     table = soup.find('table', {'class': 'wikitable sortable'})
#     ### Collect the sectors Name from Wikipedia tables
#     sectors = []
#     for row in table.findAll('tr')[1:]:
#         sector = row.findAll('td')[3].text
#         sectors.append(sector)
#     ### Remove the newline character from list
#     sample_list = sectors
#     converted_list = []
#     for element in sample_list:
#         converted_list.append(element.strip())
#     sectors = converted_list
    
#     return sectors   

In [3]:
# ### Create a dictionary including {ticker: sector}
# if __name__ == "__main__":
#     ## Run save_sp500_sector() funtion to extract sector
#     sectors = save_sp500_sector()
#     ## Make ticker as Key and sector as Value in dictionary
#     ticker_sector_dic = {tickers[i]: sectors[i] for i in range(len(tickers))} 
#     print("length of dictionary", len(ticker_sector_dic))

### Extract Market Cap Information

In [3]:
def get_Mcaps_yf(tickers):
    """
    This function is to extract the market captalization for each input stocks
    
    Input: 
    tickers: a list of tickers of stocks
    Output:
    mktCaps: a dictionary contains {ticker : mktCap}. ticker is key and market captalization is value. 
    """
    mktCaps = {}
    for ticker in tickers:
        print("Getting data for: ", ticker)
        try:
            msft = yf.Ticker(ticker)
            mktCap = msft.info['marketCap']
            # Adding a new key value pair
            mktCaps.update( {ticker : mktCap} )
            print(ticker, 'done') # check process
        except:
            pass
    return mktCaps

def getList(dict): 
    """
    Create the function to extra the key of dictionanry into list
    
    Input:
    dict: a dictionary
    Ouput:
    a list of keys from the input dictionary
    """
    return list(dict.keys()) 

def Diff(li1, li2): 
    """
    Identify the difference between two list
    
    Input: 
    two list of tickers
    
    Output:
    a list of tickers that appears in one dictionary but doesn't appear in the other dictionary 
    """
    return (list(set(li1) - set(li2))) 

def get_Mcaps_yf_2(tickers_diff):
    """
    This function is a backup method to extract the market captalization for each stocks that are not recognized by yf.Ticker function
    
    Input: 
    tickers_diff: a list of tickers of stocks that don't extract the market capitalization from funtion get_Mcaps_yf(tickers)
    Output:
    mktCaps_diff: a dictionary contains {ticker : mktCap}. ticker is key and market captalization is value. 
    """
    mktCaps_diff = {}
    for ticker in tickers_diff:
        try:
            mktCap = Ticker(ticker).summary_detail[ticker]['marketCap']
            mktCaps_diff.update( {ticker : mktCap} )
        except:
            pass
    return mktCaps_diff

In [4]:
tickers_df = pd.read_csv(sp500_tickers_location+'sp500.csv')
tickers = list(tickers_df['Tickers'])

sectors_df = pd.read_csv(sp500_tickers_location + "sp500_sectors.csv")
ticker_sector_dic = sectors_df.set_index('ticker').T.to_dict('records')[0]

In [ ]:
### Create a dictionary {ticker: market cap}
if __name__ == "__main__":
    mktCaps = get_Mcaps_yf(tickers)
    print('mktCaps done') # check process
    tickers_yahoo = getList(mktCaps)
    print('tickers_yahoo done') # check process
    tickers_diff = Diff(tickers, tickers_yahoo)
    print('tickers_diff done') # check process
    mktCaps_diff = get_Mcaps_yf_2(tickers_diff)
    print('mktCaps_diff done') # check process
    mktCaps.update(mktCaps_diff)
    tickers_merge = getList(mktCaps) ## extract the total tickers after merge
    tickers_diff_2 = Diff(tickers, tickers_merge) ## check if there is missing tickers
    for ticker in tickers_diff_2:
        mktCaps.update({ticker : 0}) ## use 0 to replace the NAN value
    print(len(mktCaps)) # check process

Getting data for:  MMM
Getting data for:  ABT
ABT done
Getting data for:  ABBV
Getting data for:  ABMD
Getting data for:  ACN
Getting data for:  ATVI
Getting data for:  ADBE
Getting data for:  AMD
Getting data for:  AAP
Getting data for:  AES
Getting data for:  AFL
Getting data for:  A
Getting data for:  APD
Getting data for:  AKAM
Getting data for:  ALK
Getting data for:  ALB
Getting data for:  ARE
Getting data for:  ALXN
Getting data for:  ALGN
Getting data for:  ALLE
Getting data for:  LNT
Getting data for:  ALL
Getting data for:  GOOGL
Getting data for:  GOOG
Getting data for:  MO
Getting data for:  AMZN
Getting data for:  AMCR
Getting data for:  AEE
Getting data for:  AAL
Getting data for:  AEP
Getting data for:  AXP
Getting data for:  AIG
Getting data for:  AMT
Getting data for:  AWK
Getting data for:  AMP
Getting data for:  ABC
Getting data for:  AME
Getting data for:  AMGN
Getting data for:  APH
Getting data for:  ADI
Getting data for:  ANSS
Getting data for:  ANTM
Getting data

Getting data for:  ODFL
Getting data for:  OMC
Getting data for:  OKE
Getting data for:  ORCL
Getting data for:  OTIS
Getting data for:  PCAR
Getting data for:  PKG
Getting data for:  PH
Getting data for:  PAYX
Getting data for:  PAYC
Getting data for:  PYPL
Getting data for:  PNR
Getting data for:  PBCT
Getting data for:  PEP
Getting data for:  PKI
Getting data for:  PRGO
Getting data for:  PFE
Getting data for:  PM
Getting data for:  PSX
Getting data for:  PNW
Getting data for:  PXD
Getting data for:  PNC
Getting data for:  PPG
Getting data for:  PPL
Getting data for:  PFG
Getting data for:  PG
Getting data for:  PGR
Getting data for:  PLD
Getting data for:  PRU
Getting data for:  PEG
Getting data for:  PSA
Getting data for:  PHM
Getting data for:  PVH
Getting data for:  QRVO
Getting data for:  PWR
Getting data for:  QCOM
Getting data for:  DGX
Getting data for:  RL
Getting data for:  RJF
Getting data for:  RTX
Getting data for:  O
Getting data for:  REG
Getting data for:  REGN
Getti

In [ ]:
# print the market caps for all tickers from S&P500
print(mktCaps)

### Concat the market cap and sector information into dataframe

In [11]:
import pandas as pd
df = pd.DataFrame({'MktCap':pd.Series(mktCaps),'Sector':pd.Series(ticker_sector_dic)})
df.reset_index(level=0, inplace=True)

In [12]:
df.rename(columns = {'index':'Ticker'})

,Ticker,MktCap,Sector
0,A,26588786688,Health Care
1,AAL,7079245824,Industrials
2,AAP,9379105792,Consumer Discretionary
3,AAPL,1468470919168,Information Technology
4,ABBV,162946875392,Health Care
...,...,...,...
500,YUM,27473999872,Consumer Discretionary
501,ZBH,25889542144,Health Care
502,ZBRA,13715151872,Information Technology
503,ZION,5715506688,Financials


### Select top 5 tickers in each sector based on the market cap

In [13]:
def SP500_topN_Mcaps(df, n = 5):
    """
    From a dataframe that contains ticker, market caps and sector, 
    this function Select top N tickers in each sector based on the market cap
    
    Input:
    df: a dataframe that contains 'Ticker', 'MktCap' and 'Sector
    n: top n tickers you want to select
    Output:
    df_top_MktCap: 
    
    """
    df_grouped = df.groupby(['Sector'])
    df_sort= df_grouped.apply(lambda x: x.sort_values(["MktCap"],ascending=False))
    df_sort=df_sort.reset_index(drop=True).rename(columns = {'index':'Ticker'})
    df_top_MktCap = df_sort.groupby('Sector').head(n)
    return df_top_MktCap

In [14]:
df_new = SP500_topN_Mcaps(df, n = 5)
print('length of top tickers:', len(df_new))

length of top tickers: 55


In [16]:
df_new.sort_values(["MktCap"],ascending=False)
df_new.to_excel("top5company.xlsx") 

# Rank these 5 top company in their own industry sector respectively based on their most important financial metrics

## Define the dataframe

In [20]:
# if the previous document is saved as .xlsx in the same folder, import the excel
df_new = pd.read_excel('top5company.xlsx', index_col=0)
# if you have run the module above, the df_new can be displayed by running the code below
df_new.head()

In [21]:
# get the unique name of sector
df_new.Sector.unique()

array(['Communication Services', 'Consumer Discretionary',
       'Consumer Staples', 'Energy', 'Financials', 'Health Care',
       'Industrials', 'Information Technology', 'Materials',
       'Real Estate', 'Utilities'], dtype=object)

### Define All the function

In [377]:
def get_ticker(df_new, sector):
    df_sub = list(df_new[df_new['Sector'] == sector].Ticker)
    return df_sub

def get_data_balance_sheet(ticker_set, dataname):
    df_ = pd.DataFrame()
    for ticker in ticker_set: 
        df = Ticker(ticker).balance_sheet(frequency="q")[['asOfDate', dataname]]
        df = df.rename(columns = {'asOfDate':'Date', dataname:ticker})
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
        df_[ticker] = df[ticker]
    return df_

def get_data_cash_flow(ticker_set, dataname):
    df_ = pd.DataFrame()
    for ticker in ticker_set: 
        df = Ticker(ticker).cash_flow(frequency="q")[['asOfDate', dataname]]
        df = df.rename(columns = {'asOfDate':'Date', dataname:ticker})
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
        df_[ticker] = df[ticker]
    return df_

def get_rank(df, SmalltoBig = False):
    df_transposed = df.T
    df_transposed['Rank_last_quater'] = df_transposed.iloc[:,-1].rank(ascending = SmalltoBig)
    df_transposed = df_transposed.sort_values(by='Rank_last_quater', ascending= True)
    return df_transposed

def get_sales_growth_each_ticker(ticker_set, dataname):
    # df_ = pd.DataFrame()
    for ticker in ticker_set: 
        df = Ticker(ticker).income_statement(frequency="q")[['asOfDate', dataname]]
        df = df.rename(columns = {'asOfDate':'Date', dataname:ticker})
        df['Date'] = pd.to_datetime(df['Date'])
        df = df[:-1]
        df.set_index('Date', inplace=True)
        df['sales_growth'] = df.pct_change()
        # df_[ticker] = df[ticker]
        print(df)
    return None

def get_sales_growth_lastQ(ticker_set, dataname):
    df_ = pd.DataFrame()
    for ticker in ticker_set: 
        df = Ticker(ticker).income_statement(frequency="q")[['asOfDate', dataname]]
        df = df.rename(columns = {'asOfDate':'Date', dataname:ticker})
        df['Date'] = pd.to_datetime(df['Date'])
        df = df[:-1]
        df.set_index('Date', inplace=True)
        df['sales_growth'] = df.pct_change()
        df = df.tail(1).drop(columns = ticker)
        df = df.rename(columns = {'sales_growth':ticker})
        df.reset_index(level=0, inplace=True)
        df.drop(columns = ['Date'])
        df_[ticker] = df[ticker]
    return df_


def get_EBITDA_EV_each_ticker(ticker_set, dataname):
    # df_ = pd.DataFrame()
    for ticker in ticker_set: 
        df = Ticker(ticker).income_statement(frequency="q")[['asOfDate', dataname]]
        df = df.rename(columns = {'asOfDate':'Date', dataname:ticker})
        df['Date'] = pd.to_datetime(df['Date'])
        df = df[:-1]
        df.set_index('Date', inplace=True)
        df = df.dropna()
        ev = Ticker(ticker).key_stats[ticker]["enterpriseValue"]
        print("{} has enterprice value of {}".format(ticker,ev))
        df['EBITDA_EV'] = df[ticker].div(ev)
        print("{} has historical NormalizedEBITDA and EBITDA/EV:".format(ticker))
        print(df)
        print('------------------------------')     
    return None

def get_EBITDA_EV(ticker_set, dataname):
    df_ = pd.DataFrame()
    for ticker in ticker_set: 
        df = Ticker(ticker).income_statement(frequency="q")[['asOfDate', dataname]]
        df = df.rename(columns = {'asOfDate':'Date', dataname:ticker})
        df['Date'] = pd.to_datetime(df['Date'])
        df = df[:-1]
        df.set_index('Date', inplace=True)
        df = df.dropna()
        ev = Ticker(ticker).key_stats[ticker]["enterpriseValue"]
        df['EBITDA_EV'] = df[ticker].div(ev)
        df = df.drop(columns = ticker)
        df = df.rename(columns = {'EBITDA_EV':ticker})
        df_[ticker] = df[ticker]
    return df_

def get_EBITDA_EV_lastQ(ticker_set, dataname):
    df_ = pd.DataFrame()
    for ticker in ticker_set: 
        df = Ticker(ticker).income_statement(frequency="q")[['asOfDate', dataname]]
        df = df.rename(columns = {'asOfDate':'Date', dataname:ticker})
        df['Date'] = pd.to_datetime(df['Date'])
        df = df[:-1]
        df.set_index('Date', inplace=True)
        df = df.dropna()
        ev = Ticker(ticker).key_stats[ticker]["enterpriseValue"]
        df['EBITDA_EV'] = df[ticker].div(ev)
        df = df.tail(1).drop(columns = ticker)
        df = df.rename(columns = {'EBITDA_EV':ticker})
        df.reset_index(level=0, inplace=True)
        df.drop(columns = ['Date'])
        df_[ticker] = df[ticker]
    return df_

def get_data_income_statement_deletelastrow(ticker_set, dataname):
    df_ = pd.DataFrame()
    for ticker in ticker_set: 
        df = Ticker(ticker).income_statement(frequency="q")[['asOfDate', dataname]]
        df = df.rename(columns = {'asOfDate':'Date', dataname:ticker})
        df['Date'] = pd.to_datetime(df['Date'])
        df = df[:-1]
        df.set_index('Date', inplace=True)
        df_[ticker] = df[ticker]
    return df_

## Health Care 
### [Cash&Cash Equivalent, FCF]

### Get Cash and Cash Equivalent Data

In [108]:
if __name__ == "__main__":
    ticker_set = get_ticker(df_new, 'Health Care')
    df_cash = get_data_balance_sheet(ticker_set,'CashAndCashEquivalents')
    print("Historical Data for Cash and cash equivalents")
    print(df_cash)
    df_rank = get_rank(df_cash)
    print("----------------------------------------------")
    print("Rank for Cash and cash equivalents")
    print(df_rank)

Historical Data for Cash and cash equivalents
                     JNJ           UNH           MRK           PFE  \
Date                                                                 
2019-03-31  1.473400e+10  1.240700e+10  8.076000e+09  1.937000e+09   
2019-06-30  1.437600e+10  1.374500e+10  6.659000e+09  1.784000e+09   
2019-09-30  1.624900e+10  1.236300e+10  7.869000e+09  2.785000e+09   
2019-12-31  1.730500e+10  1.098500e+10  9.676000e+09  1.305000e+09   
2020-03-31  1.553000e+10  2.156900e+10  7.425000e+09  2.151000e+09   

                    ABBV  
Date                      
2019-03-31  4.897000e+09  
2019-06-30  5.172000e+09  
2019-09-30  1.064800e+10  
2019-12-31  3.992400e+10  
2020-03-31  4.114200e+10  
----------------------------------------------
Rank for Cash and cash equivalents
Date  2019-03-31 00:00:00  2019-06-30 00:00:00  2019-09-30 00:00:00  \
ABBV         4.897000e+09         5.172000e+09         1.064800e+10   
UNH          1.240700e+10         1.374500e+10    

## Get Free Cash Flow Data

In [151]:
if __name__ == "__main__":
    ticker_set = get_ticker(df_new, 'Health Care')
    df_cash = get_data_cash_flow(ticker_set, 'FreeCashFlow')
    print("Historical Data for Free Cash Flow")
    print(df_cash)
    df_rank = get_rank(df_cash)
    print("----------------------------------------------")
    print("Rank for Free Cash Flow")
    print(df_rank)

Historical Data for Free Cash Flow
                     JNJ           UNH           MRK           PFE  \
Date                                                                 
2019-03-31  2.887000e+09  2.672000e+09  7.410000e+08  1.080000e+09   
2019-06-30  5.111000e+09  5.459000e+09  2.302000e+09  2.023000e+09   
2019-09-30  6.782000e+09  2.706000e+09  3.267000e+09  3.848000e+09   
2019-12-31  5.138000e+09  5.555000e+09  3.657000e+09  3.043000e+09   
2020-03-31  2.733000e+09  2.474000e+09 -2.790000e+08  2.638000e+09   
2020-03-31  1.976400e+10  1.619400e+10  8.947000e+09  1.155200e+10   

                    ABBV  
Date                      
2019-03-31  2.910000e+09  
2019-06-30  2.349000e+09  
2019-09-30  4.401000e+09  
2019-12-31  3.112000e+09  
2020-03-31  3.690000e+09  
2020-03-31  1.355200e+10  
----------------------------------------------
Rank for Free Cash Flow
Date  2019-03-31 00:00:00  2019-06-30 00:00:00  2019-09-30 00:00:00  \
JNJ          2.887000e+09         5.111000e+09

## Consumer Discretionary 
### [Sales growth]

### Sales Growth

In [286]:
if __name__ == "__main__":
    ticker_set = get_ticker(df_new,'Consumer Discretionary')
    print("-------------------------------------")
    print("Historical Total Revenue and Sales Growth Data For 5 Ticker")
    get_sales_growth_each_ticker(ticker_set, 'TotalRevenue')

-------------------------------------
Historical Total Revenue and Sales Growth Data For 5 Ticker
                    AMZN  sales_growth
Date                                  
2019-03-31  5.970000e+10           NaN
2019-06-30  6.340400e+10      0.062044
2019-09-30  6.998100e+10      0.103732
2019-12-31  8.743600e+10      0.249425
2020-03-31  7.545200e+10     -0.137060
                      HD  sales_growth
Date                                  
2019-04-30  2.638100e+10           NaN
2019-07-31  3.083900e+10      0.168985
2019-10-31  2.722300e+10     -0.117254
2020-01-31  2.578200e+10     -0.052933
2020-04-30  2.826000e+10      0.096114
                     NKE  sales_growth
Date                                  
2019-05-31  1.018400e+10           NaN
2019-08-31  1.066000e+10      0.046740
2019-08-31  3.982900e+10      2.736304
2019-11-30  1.032600e+10     -0.740742
2020-02-29  1.010400e+10     -0.021499
                     MCD  sales_growth
Date                                  
2019-

In [287]:
if __name__ == "__main__":
    ticker_set = get_ticker(df_new,'Consumer Discretionary')
    df_sales_g = get_sales_growth_lastQ(ticker_set, 'TotalRevenue')
    df_rank = get_rank(df_sales_g)
    df_rank = df_rank.rename(columns = {0:'sales_growth'})
    print("-------------------------------------")
    print("Rank for Sales Growth in Last Quarter")
    print(df_rank)

-------------------------------------
Rank for Sales Growth in Last Quarter
      sales_growth  Rank_last_quater
LOW       0.227616               1.0
HD        0.096114               2.0
NKE      -0.021499               3.0
MCD      -0.118639               4.0
AMZN     -0.137060               5.0


## Energy
### EBITDA/EV

In [378]:
if __name__ == "__main__":
    ticker_set = get_ticker(df_new, 'Energy')
    print('---------- Calculate the EBITDA_EV -------')
    get_EBITDA_EV_each_ticker(ticker_set,'NormalizedEBITDA')
    df = get_EBITDA_EV(ticker_set, 'NormalizedEBITDA')
    print(df)
    df_rank = get_rank(df)
    print("----------------------------------------------")
    print("Rank for EBITDA_EV")
    print(df_rank)

---------- Calculate the EBITDA_EV -------
XOM has enterprice value of 254308728832
XOM has historical NormalizedEBITDA and EBITDA/EV:
                     XOM  EBITDA_EV
Date                               
2019-03-31  9.041000e+09   0.035551
2019-06-30  9.479000e+09   0.037274
2019-09-30  9.826000e+09   0.038638
2019-12-31  1.153800e+10   0.045370
2020-03-31  5.810000e+09   0.022846
------------------------------
CVX has enterprice value of 197283151872
CVX has historical NormalizedEBITDA and EBITDA/EV:
                     CVX  EBITDA_EV
Date                               
2019-03-31  8.276000e+09   0.041950
2019-06-30  1.046700e+10   0.053056
2019-09-30  8.607000e+09   0.043628
2019-12-31  8.202000e+09   0.041575
2020-03-31  8.595000e+09   0.043567
------------------------------
COP has enterprice value of 53748154368
COP has historical NormalizedEBITDA and EBITDA/EV:
                     COP  EBITDA_EV
Date                               
2019-03-31  4.462000e+09   0.083017
2019-06-

## Consumer Staples
### EBITDA/EV

In [361]:
if __name__ == "__main__":
    ticker_set = get_ticker(df_new, 'Consumer Staples')
    print('---------- Calculate the EBITDA_EV -------')
    get_EBITDA_EV_each_ticker(ticker_set,'NormalizedEBITDA')
    print("----------------------------------------------")
    df = get_EBITDA_EV_lastQ(ticker_set,'NormalizedEBITDA')
    print("-------------------------------------")
    print('Show the data for last quater')
    print(df)
    df_rank = get_rank(df)
    df_rank = df_rank.rename(columns = {0:'EBITDA_EV'})
    print("-------------------------------------")
    print("Rank for EBITDA_EV in Last Quarter")
    print(df_rank)

---------- Calculate the EBITDA_EV -------
WMT has enterprice value of 400961208320
WMT has historical NormalizedEBITDA and EBITDA/EV:
                     WMT  EBITDA_EV
Date                               
2019-04-30  8.544000e+09   0.021309
2019-07-31  8.276000e+09   0.020640
2019-10-31  7.729000e+09   0.019276
2020-01-31  9.153000e+09   0.022828
2020-04-30  8.779000e+09   0.021895
------------------------------
PG has enterprice value of 310413787136
PG has historical NormalizedEBITDA and EBITDA/EV:
                      PG  EBITDA_EV
Date                               
2019-03-31  4.120000e+09   0.013273
2019-06-30 -4.134000e+09  -0.013318
2019-09-30  5.174000e+09   0.016668
2019-12-31  5.309000e+09   0.017103
2020-03-31  4.397000e+09   0.014165
------------------------------
KO has enterprice value of 230261047296
KO has historical NormalizedEBITDA and EBITDA/EV:
                      KO  EBITDA_EV
Date                               
2019-03-31  2.639000e+09   0.011461
2019-06-30 

## 'Utilities': 
### Debt-To-Equity (D/E) Ratios

In [379]:
if __name__ == "__main__":
    ticker_set = get_ticker(df_new, 'Utilities')
    df_equity = get_data_balance_sheet(ticker_set, 'TotalEquityGrossMinorityInterest')
    df_debt = get_data_balance_sheet(ticker_set, 'TotalDebt')
    df_DtoE = df_debt / df_equity
    print(df_equity)
    print(df_debt)
    print(df_DtoE)
    df_rank = get_rank(df_DtoE, SmalltoBig = True)
    print("----------------------------------------------")
    print("Rank for Debt to Equity")
    print(df_rank)

                     NEE             D           DUK            SO  \
Date                                                                 
2019-03-31  3.784000e+10  2.759900e+10  4.504500e+10  3.097800e+10   
2019-06-30  3.842600e+10  2.838100e+10  4.533200e+10  3.142200e+10   
2019-09-30  4.017400e+10  2.995700e+10  4.740700e+10  3.208200e+10   
2019-12-31  4.136000e+10  3.403300e+10  4.795100e+10  3.205000e+10   
2020-03-31  4.087500e+10  3.275200e+10  4.808300e+10  3.220700e+10   

                     AEP  
Date                      
2019-03-31  1.922850e+10  
2019-06-30  1.942320e+10  
2019-09-30  1.999770e+10  
2019-12-31  1.991320e+10  
2020-03-31  2.000770e+10  
                     NEE             D           DUK            SO  \
Date                                                                 
2019-03-31  4.021300e+10  4.277100e+10  6.069900e+10  4.596900e+10   
2019-06-30  4.171700e+10  4.196300e+10  6.233500e+10  4.615800e+10   
2019-09-30  4.178700e+10  4.127300e+10  

## 'Financials': 
### Return on Equity (ROE)

In [429]:
if __name__ == "__main__":
    ticker_set = get_ticker(df_new, 'Financials')
    df_equity = get_data_balance_sheet(ticker_set, 'StockholdersEquity')
    df_income = get_data_cash_flow(ticker_set, 'NetIncome')
    df_income = df_income[:-1]
    print("----------------------------------------------")
    print("Historical Data of Share Holder Equity")
    print(df_equity)
    print("----------------------------------------------")
    print("Historical Data of Net Income")
    print(df_income)
    df_ItoE = df_income / df_equity
    print("----------------------------------------------")
    print("Historical Data of Return on Equity")
    print(df_ItoE)
    df_rank = get_rank(df_ItoE, SmalltoBig = False)
    print("----------------------------------------------")
    print("Rank for Return on Equity: The higher the better")
    print(df_rank)

----------------------------------------------
Historical Data of Share Holder Equity
                   BRK-B           JPM           BAC           WFC  \
Date                                                                 
2019-03-31  3.688770e+11  2.598370e+11  2.670100e+11  1.978320e+11   
2019-06-30  3.825440e+11  2.632150e+11  2.714080e+11  1.990420e+11   
2019-09-30  3.976090e+11  2.643480e+11  2.683870e+11  1.933040e+11   
2019-12-31  4.247910e+11  2.613300e+11  2.648100e+11  1.871460e+11   
2020-03-31  3.715650e+11  2.612620e+11  2.649180e+11  1.827180e+11   

                       C  
Date                      
2019-03-31  1.962520e+11  
2019-06-30  1.973590e+11  
2019-09-30  1.963730e+11  
2019-12-31  1.932420e+11  
2020-03-31  1.923310e+11  
----------------------------------------------
Historical Data of Net Income
                   BRK-B           JPM           BAC           WFC  \
Date                                                                 
2019-03-31  2.166

## 'Communication Services': 
### P/E ratio | EPS growth

### EPS growth

In [428]:
if __name__ == "__main__":
    ticker_set = get_ticker(df_new, 'Communication Services')
    df = get_data_income_statement_deletelastrow(ticker_set, 'BasicEPS')
    print('Historical Basic EPS data')
    print(df)
    df_change = df.pct_change()
    print("-------------------------------------")
    print('Historical Basic EPS growth data')
    print(df_change)
    print("-------------------------------------")
    print("Rank for Basic EPS growth in Last Quarter: the higher growth rate the better")
    df_rank = get_rank(df_change, SmalltoBig = False)
    print(df_rank)

Historical Basic EPS data
            GOOGL   GOOG    FB    VZ     T
Date                                      
2019-03-31   9.58   9.58  0.85  1.22  0.56
2019-06-30  14.33  14.33  0.92  0.95  0.51
2019-09-30  10.20  10.20  2.13  1.26  0.50
2019-12-31  15.49  15.49  2.58  1.23  0.33
2020-03-31   9.96   9.96  1.72  1.00  0.63
-------------------------------------
Historical Basic EPS growth data
               GOOGL      GOOG        FB        VZ         T
Date                                                        
2019-03-31       NaN       NaN       NaN       NaN       NaN
2019-06-30  0.495825  0.495825  0.082353 -0.221311 -0.089286
2019-09-30 -0.288207 -0.288207  1.315217  0.326316 -0.019608
2019-12-31  0.518627  0.518627  0.211268 -0.023810 -0.340000
2020-03-31 -0.357005 -0.357005 -0.333333 -0.186992  0.909091
-------------------------------------
Rank for Basic EPS growth in Last Quarter: the higher growth rate the better
Date   2019-03-31 00:00:00  2019-06-30 00:00:00  2019-09-30 